# CNN-Based Classification of Cats vs. Dogs from Image Data

This project leverages a convolutional neural network (CNN) to classify images of dogs and cats. The dataset was obtained from Kaggle and preprocessed for use in a deep learning model built with TensorFlow and Keras.

The project demonstrates the complete workflow of loading and preparing image data, building a CNN architecture, applying data augmentation and regularization techniques, and evaluating model performance using accuracy and loss visualizations.

Dataset Details: https://www.kaggle.com/datasets/salader/dogs-vs-cats


In [ ]:
import os
import zipfile
import subprocess
import shutil
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import RandomFlip, RandomRotation, RandomZoom
from keras.models import Sequential as KerasSequential
import matplotlib.pyplot as plt

# 📁 Dataset Preparation: Downloading and Extracting Kaggle Dataset


We begin by setting up Kaggle API access to download the 'Dogs vs. Cats' dataset. We use Python’s built-in libraries to authenticate and download the ZIP archive, which is then extracted for use in model training.

Steps:
1. Copy `kaggle.json` to the appropriate directory.
2. Download the dataset using `kaggle datasets download`.
3. Unzip the downloaded file for training and validation usage.


In [ ]:
# Set up Kaggle
kaggle_json = 'kaggle.json'
kaggle_dir = os.path.expanduser('~/.kaggle')

if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

# Copy and set permissions
shutil.copy(kaggle_json, os.path.join(kaggle_dir, 'kaggle.json'))
os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)

# Download the dataset using Kaggle
print("Downloading dataset from Kaggle...")
subprocess.run(['kaggle', 'datasets', 'download', '-d', 'salader/dogs-vs-cats'], check=True)

# Unzip the downloaded dataset
zip_path = 'dogs-vs-cats.zip'
extract_path = 'dogs-vs-cats'

print(f"Unzipping dataset to: {extract_path}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Download and extraction complete!")

# ⚙️ Configuration & Data Loading


We configure important parameters which includes batch size, image dimensions, and the number of epochs. We use `image_dataset_from_directory` to load images from the extracted dataset directory.

In [ ]:
# -------------------------------
# CONFIG
# -------------------------------
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)
TRAIN_DIR = os.path.join("dogs-vs-cats", "train")
TEST_DIR = os.path.join("dogs-vs-cats", "test")
EPOCHS = 10
AUTOTUNE = tf.data.AUTOTUNE

# -------------------------------
# LOAD DATA
# -------------------------------
def load_dataset(directory):
    return keras.utils.image_dataset_from_directory(
        directory=directory,
        labels='inferred',
        label_mode='int',
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE
    )

print("Loading training dataset...")
train_ds = load_dataset(TRAIN_DIR)

print("Loading validation dataset...")
validation_ds = load_dataset(TEST_DIR)

# 🧼 Preprocessing: Normalization

Pixel values are normalized to the range [0, 1] for better numerical stability during training. Data augmentation is applied to the training data to reduce overfitting by randomly flipping and rotating images.

In [ ]:
# -------------------------------
# DATA AUGMENTATION & NORMALIZATION
# -------------------------------
data_augmentation = KerasSequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
])

def preprocess(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y)).map(preprocess)
validation_ds = validation_ds.map(preprocess)

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.prefetch(buffer_size=AUTOTUNE)

# 🧠 Model Architecture: CNN

Defined a convolutional neural network with three convolutional layers, each followed by batch normalization and max pooling.

Dense layers follow the feature extractor, ending in a sigmoid layer for binary classification.

Improvements for generalization:
- Dropout layers
- BatchNormalization

In [ ]:
# -------------------------------
# BUILD CNN MODEL
# -------------------------------
model = Sequential([
    Input(shape=(256, 256, 3)),

    Conv2D(32, (3, 3), activation='relu', padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'),

    Conv2D(64, (3, 3), activation='relu', padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'),

    Conv2D(128, (3, 3), activation='relu', padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

model.summary()

# 🛠️ Compile & Train the Model

Compiled the model with the Adam optimizer and binary cross-entropy loss.
Early stopping is implemented to halt training if validation loss does not improve, helping reduce overfitting.

In [ ]:
# -------------------------------
# COMPILE
# -------------------------------
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# -------------------------------
# CALLBACKS
# -------------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', verbose=1)
]

# -------------------------------
# TRAIN MODEL
# -------------------------------
print("Starting training with augmentation and callbacks...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=validation_ds,
    callbacks=callbacks
)

# 📈 Visualization: Accuracy and Loss Trends

Plotting training and validation accuracy/loss across epochs allows us to visually inspect model performance and check for overfitting.

In [ ]:
# -------------------------------
# VISUALIZE PERFORMANCE
# -------------------------------
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy', color='red')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss', color='red')
plt.plot(history.history['val_loss'], label='Validation Loss', color='blue')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

### Discussion & Conclusion

This project aimed to classify images of cats and dogs using a convolutional neural network (CNN), demonstrating how model performance can be improved through strategic enhancements and evaluated through key metrics such as accuracy and loss.

The model initially started with modest accuracy and high loss, indicating room for optimization. To improve performance, we implemented several key strategies:
- **Data Augmentation** was applied to increase dataset variability and reduce overfitting.
- **Batch Normalization** layers were added to stabilize and accelerate training.
- **Dropout** was introduced to reduce the risk of overfitting.

The **accuracy plot** reveals a steady increase in training accuracy over epochs, with validation accuracy initially improving in parallel. However, fluctuations in validation accuracy, especially after epoch 7, suggest possible overfitting or instability in generalization.

The **loss plot** corroborates this interpretation. Training loss decreased consistently, indicating effective learning, while the validation loss showed a rise and variability after epoch 6. This divergence is a typical sign of overfitting, where the model learns the training data too specifically and struggles to generalize.

In summary, this project provided entry experience in designing and refining a CNN model for image classification. By monitoring performance curves and iteratively adjusting architecture and hyperparameters, we observed tangible improvements in model behavior and learned key practices in deep learning model development and evaluation.